In [4]:
#
# Copyright (C) 2024 by Sonja Filiposka <sonja.filiposka@finki.ukim.mk>
#
# This code is licensed under a Creative Commons Attribution 4.0 International License. (see LICENSE.txt for details)
#
# General Description - this notebook is used to extract several attributes information from OMNET output vector file
# It creates an output file that contains values of a set of attributes, named "distance", "measuredSinrDl", "measuredSinrUl", 
#   "rcvdSinrDl", "averageCqiDl", "servingCell", "rlcDelayDl", "rlcPacketLossTotal", "rlcPduDelayDl", "rlcPduPacketLossDl",
#   "rlcPduThroughputDl", "rlcThroughputDl", "receivedPacketFromLowerLayer" from OMNeT++ output vectors
#

time: 403 µs


In [5]:
def dataset_omnet(vector_omnet, output):

    # parameters
    # 1 = $vector_omnet - omnet input.file name
    # 2 = $output       - output.file name
        
    # First prepare the output file from OMNET, vector-0.vec to suit version 2.0
    !split -d -b 500m $vector_omnet
    !sed -i 's/version 3/version 2/' x00
    !cat x* > vector-1.vec
    # clean
    !rm x*
    # requires to have netperfmeter installed: sudo apt-get install netperfmeter
    !extractvectors vector-1.vec results.bz2 "distance" \
        "measuredSinrDl" "measuredSinrUl" "rcvdSinrDl" "averageCqiDl" \
        "servingCell" "rlcDelayDl" "rlcPacketLossTotal" "rlcPduDelayDl" \
        "rlcPduPacketLossDl" "rlcPduThroughputDl" "rlcThroughputDl"\
        "receivedPacketFromLowerLayer" 

    # get data file: results (it deletes results.bz2)
    if os.path.isfile('results'):
        !rm results
    !bzip2 -d results.bz2
    
    # removing unnecessary information
    !grep 'car' results > r_cars
    !sed -i -e 's/NRSeveralBSALC.//' -e 's/:vector//'  -e 's/ ETV//' \
        -e 's/(packetBytes)//' r_cars
    
    # clean
    !rm vector-1.vec 
    
    cols=['Time','Event','Object','Vector','Split','Value']
    data = vx.read_csv("r_cars", sep='\t', header=None,
        #names=['#','Time','Event','Object','Vector','Split','Value'],
        names=['#','Time','Event','Object','Vector','Value'],
        usecols=['Time','Object','Vector','Value'],
        convert=True, chunk_size=150_000_000) 
    print(data)

    #data = data['#'].drop
    data.Object = data.Object.str.replace('"', '')
    data.Vector = data.Vector.str.replace('"', '')    

    data.export_csv(output, index=False, sep='\t')
    
    # clean
    !rm results r_cars* 
    del data
    
    return 1 


time: 811 µs


In [6]:
# measuring execution time
%load_ext autotime

# extract the delay and the handover information from omnet output vector file
maxTime = 1800
communities = 9

#parametrised calls of notebook
import numpy as np
import pandas as pd
import vaex as vx
import os.path

path="/home/jupyter/notebook/OMNET6.0/"
sumo_files = "dataset_AI_input/fdc_signals_"
omnet_files = "VoipDl-Urban-xxxx/VoipDl-Urban-"
#cars = np.array([4928, 4951, 4955, 5712, 5734, 5749, 6900, 6908, 6923, 8589, 8619, 8620])
cars = np.array([4928])
out = "dataset_AI_output/"
outFile = "_AI.csv"
# initialPositioning-xxxx.txt and migrations-xxxx.txt will also be.ipynb_checkpoints/created

for i in cars:
    # create omnet dataframe
    v_omnet = path + omnet_files + str(i) + "/vector-0.vec"
    output = path + out + str(i) + "_omnet" + outFile
    print("STEP 1) omnet input : ", v_omnet)
    print("STEP 1) omnet output : ", output)
    # extract dataset from OMNET vector
    omnet_exit_code1 = dataset_omnet(v_omnet, output)
    print(omnet_exit_code1) 

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
STEP 1) omnet input :  /home/jupyter/notebook/OMNET6.0/VoipDl-Urban-xxxx/VoipDl-Urban-4928/vector-0.vec
STEP 1) omnet output :  /home/jupyter/notebook/OMNET6.0/dataset_AI_output/4928_omnet_AI.csv
ExtractVectors - Version 1.30

Adding vector "servingCell:vector ETV" of object NRSeveralBSALC.car[0].cellularNic.phy ...
Adding vector "distance:vector ETV" of object NRSeveralBSALC.car[0].cellularNic.nrChannelModel[0] ...
Adding vector "servingCell:vector ETV" of object NRSeveralBSALC.car[0].cellularNic.nrPhy ...
Adding vector "measuredSinrUl:vector ETV" of object NRSeveralBSALC.car[0].cellularNic.nrChannelModel[0] ...
Adding vector "measuredSinrDl:vector ETV" of object NRSeveralBSALC.car[0].cellularNic.nrChannelModel[0] ...
Adding vector "receivedPacketFromLowerLayer:vector(packetBytes) ETV" of object NRSeveralBSALC.gNodeB2.cellularNic.mac ...
Adding vector "averageCqiDl:vector ETV" of object NRSeveralBSALC.